In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[  2136.06674665,   2008.03265958],
        [  2775.0543623 ,   5468.17490943],
        [  3033.91173729,   3047.60914358],
        [  5158.83944197,   5394.98263971],
        [  3182.06559307,   3444.44727344],
        [  9103.90874818,   8867.68405924],
        [  2678.65145695,   2640.19073646],
        [  2748.06623017,   3296.74133768],
        [  6257.27450378,   8436.71914537],
        [ 10663.45670255,  10991.55537808],
        [  7545.06461598,   8896.28419843],
        [  8125.21554502,   8728.30233267],
        [  7507.71498527,   6131.11623836],
        [  3010.41299015,   1051.25994621],
        [  6769.72409266,   3680.54976191],
        [  1517.99174628,   1984.67118849],
        [  2701.27571664,   3095.10067561],
        [  1788.12023883,   2238.52827238],
        [ 12473.87484785,  13375.45259996],
        [  2315.29324646,   2472.40915216],
        [ 10656.4209806 ,  11190.66546003],
        [  4000.83198032,   4325.07903563],
        [  2582.1359908 ,   3037

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 5e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.23059331e+06  0.00000000e+00  5.46e+03 3.50e+02  8.85e+05     0s
   1   2.26668545e+05 -2.32656176e+06  9.02e+02 1.45e+02  1.98e+05     0s
   2   1.56258760e+05 -1.22975192e+06  9.02e-04 1.14e-13  2.48e+04     0s
   3  -2.76696531e+05 -4.67404470e+05  9.02e-10 5.68e-14  3.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.121259195111292e-09,
 2894.6714498790243,
 1.2307693947690642e-09,
 1.646428286292278e-05,
 1213.6970674937872,
 2.7628826312128205e-05,
 3.6649863432705777e-07,
 0.011476826316431307,
 355.77764443727426,
 3381.5160975866584,
 1.6462553713742113e-05,
 638.3246923113089,
 2.762877322756642e-05,
 2.7786900230847273e-05,
 16.695979990776863,
 5385.764318340252,
 1.6465053923867337e-05,
 1.646504732608081e-05,
 1008.1149894457322,
 11429.438516742248,
 2.7622923075660763e-05,
 2154.6844147730963,
 1423.5413108337095,
 7486.569079876292,
 30.934765275311747,
 17.11335846529238,
 309.852835879974,
 2933.718409161956,
 395.5084641124031,
 927.5654921444665,
 846.1595550965931,
 380.5066213224944,
 6862.898070776836,
 1995.695417142654,
 3423.4968307483277,
 1629.66945580147,
 267.53039851409403,
 5153.2660246307905,
 1407.3595334526822,
 1061.6339239812298,
 6318.434073356713,
 752.2126269832681,
 4294.920859733498,
 36.076809899284285,
 27.697566123399113,
 0.0004691224099180728,
 0.0004